In [1]:
import GST
from GSTCommons import FiducialPairReduction as FPR

In [2]:
import json

# Follow Algorithm tutorial to generate LSGST gatesets
gs_target = GST.loadGateset("tutorial_files/Example_Gateset.txt")
ds = GST.loadDataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = GST.loadGatestringList("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = GST.getRhoAndESpecs(fiducialGateStrings=fiducialList)
gs_lgst = GST.doLGST(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = GST.optimizeGauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = GST.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of LSGST to use
specs  = GST.getRhoAndESpecs(fiducialGateStrings=fiducialList)
germList = GST.loadGatestringList("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))

Loading from cache file:  tutorial_files/Example_Dataset.txt.cache


In [3]:
#Get sufficient set of fiducial pairs, meaning that with these fidicual pairs and the given set of germs,
# the number of gateset parameters which are amplified when all pairs are used are also amplified when using
# the returned subset.
rhoEPairs = FPR.getSufficientFiducialPairs(gs_target, fiducialList, fiducialList, germList, verbosity=1)
print rhoEPairs

------  Fiducial Pair Reduction --------
maximum number of amplified parameters =  34
Beginning search for a good set of 1 pairs (36 pair lists to test)
 --> Higheset number of amplified parameters was 19
Beginning search for a good set of 2 pairs (630 pair lists to test)
 --> Higheset number of amplified parameters was 30
Beginning search for a good set of 3 pairs (7140 pair lists to test)
[(0, 0), (0, 1), (1, 0)]


In [4]:
#Test a specific set of fiducial pairs: see how many gateset parameter are amplified
#FPR.getSufficientFiducialPairs(gs_target, fiducialList, germList, testPairList=[(0,0),(0,1),(1,0)], verbosity=4)

In [5]:
from GSTCommons import MakeLists_WholeGermPowers as MakeLists
lsgstListOfLists = MakeLists.make_lsgst_lists(gs_target.keys(), fiducialList, germList,
                                              maxLengthList, rhoEPairs)

gs_lsgst_list = GST.Core.doIterativeLSGST(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )


--- Iterative LSGST: Beginning iter 1 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 56 model params = expected mean of 36)

--- Iterative LSGST: Beginning iter 2 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 56 model params = expected mean of 36)

--- Iterative LSGST: Beginning iter 3 of 10 : 95 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 51.0054 (95 data params - 56 model params = expected mean of 39)

--- Iterative LSGST: Beginning iter 4 of 10 : 114 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 76.4723 (114 data params - 56 model params = expected mean of 58)

--- Iterative LSGST: Beginning iter 5 of 10 : 146 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 117.016 (146 data params - 56 model params = expected mean of 90)

--- Iterative LSGST: Beginning iter 6 of 10 : 178 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 145.915 (1

In [6]:
# Compute a few additional quantities needed to generate the report
Ls = maxLengthList
gateStrDict = { (L,germ):GST.GateStringTools.repeatWithMaxLength(germ,L,False) for L in Ls for germ in germList }

#remove duplicates by replacing duplicate strings with None
runningList = []
for L in Ls:
    for germ in germList:
        if gateStrDict[(L,germ)] in runningList:
            gateStrDict[(L,germ)] = None
        else: runningList.append( gateStrDict[(L,germ)] )

In [7]:
import GST.ReportGeneration as RG
reload(RG)
RG.writeReport_LsAndGerms("tutorial_files/Example_report_FR.pdf","chi2",maxLengthList[1:],gs_lsgst_list[1:],
                          lsgstListOfLists[1:], ds, "$\\mathcal{D}_0$", germList, gateStrDict,
                          fiducialList, fiducialList, gs_target, "An Example GST Report",m=0,M=10,
                          appendices=False, rhoEPairs = rhoEPairs)

Generating target gateset tables...
Generating dataset and gatestring list tables...
Generating spam estimate tables...
Generating gate estimate table...
Generating choi matrix table...
Generating gate decomposition tables...
Generating closest unitary table...
Generating GST estimate vs. target tables...
Generating chi2 progress tables...
Generating plots...
 -- Chi2 plots (3):  1  2  3  
 -- Blank Box plots...
 -- Direct-X plots  (4):
 1  2  3  4
 -- Error rate plots...
Merging into template file...
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial output PDF tutorial_files/Example_report_FR.pdf successfully generated.
Final output PDF tutorial_files/Example_report_FR.pdf successfully generated. Cleaning up .aux and .log files.
